# Data Extraction and Conversion from netCDF4 to Pandas DataFrame

This code extracts temperature (t2m) and total precipitation (tp) data from a netCDF4 file and converts it into a Pandas DataFrame. Here are the steps:

1. Import necessary libraries:
   - netCDF4: for working with netCDF files.
   - Pandas: for creating and manipulating DataFrames.
   - NumPy: for numerical operations.

2. Open the netCDF file and print its dimensions and variables.

3. Extract temperature (t2m) and total precipitation (tp) variables along with their attributes.

4. Extract scale factor and add offset for both temperature and total precipitation.

5. Convert time values to datetime objects using the num2date function.

6. Apply scale factor and add offset for temperature and total precipitation to obtain actual values.

7. Convert temperature from Kelvin to degrees Celsius.

8. Convert total precipitation from meters to millimeters.

9. Create arrays for latitude, longitude, and experiment version.

10. Create 2D arrays for latitude, longitude, and experiment version using meshgrid.

11. Flatten the 2D arrays for easier DataFrame creation.

12. Create a DataFrame containing time, latitude, longitude, experiment version, temperature (t2m), and total precipitation (tp) values.

13. Save the DataFrame to a CSV file named "climate_data_all_rows.csv".

14. Print a confirmation message indicating that the data has been saved successfully.



In [ ]:
import netCDF4 as nc
from netCDF4 import num2date
import pandas as pd
import numpy as np

In [ ]:
df = nc.Dataset('ERA5_Reanalysis.nc')
print(df.dimensions)

In [ ]:
print(df.variables)

In [ ]:
# Open the netCDF file
with df as dataset:
    # Access variables and their attributes
    t2m_variable = dataset.variables['t2m']
    tp_variable = dataset.variables['tp']
    time_variable = dataset.variables['time']
    expver_variable = dataset.variables['expver']
    lat_variable = dataset.variables['latitude']
    lon_variable = dataset.variables['longitude']

    # Extract scale factor and add offset for temperature
    t2m_scale_factor = t2m_variable.scale_factor
    t2m_add_offset = t2m_variable.add_offset

    # Extract scale factor and add offset for total precipitation
    tp_scale_factor = tp_variable.scale_factor
    tp_add_offset = tp_variable.add_offset

    # Convert time values to datetime objects
    times = num2date(time_variable[:], units=time_variable.units)

    # Apply scale factor and add offset for temperature and total precipitation
    t2m_values = t2m_variable[:] * t2m_scale_factor + t2m_add_offset
    tp_values = tp_variable[:] * tp_scale_factor + tp_add_offset

    # Convert temperature from Kelvin to degrees Celsius
    t2m_values_celsius = t2m_values - 273.15

    # Convert total precipitation from meters to millimeters
    tp_values_mm = tp_values * 1000  # 1 meter = 1000 millimeters

    # Create 1D arrays for latitude, longitude, and expver
    latitudes = lat_variable[:]
    longitudes = lon_variable[:]
    expver_values = expver_variable[:]

    # Create 2D arrays for latitude, longitude, and expver
    latitudes_2d, longitudes_2d, expver_2d = np.meshgrid(latitudes, longitudes, expver_values, indexing='ij')

    # Flatten the 2D arrays for easier DataFrame creation
    latitudes_flat = latitudes_2d.flatten()
    longitudes_flat = longitudes_2d.flatten()
    expver_flat = expver_2d.flatten()

    # Create DataFrame with time, latitude, longitude, expver, t2m, and tp values
    df = pd.DataFrame({
        'time': np.repeat(times, len(latitudes_flat)),
        'latitude': np.tile(latitudes_flat, len(times)),
        'longitude': np.tile(longitudes_flat, len(times)),
        'expver': np.tile(expver_flat, len(times)),
        't2m': t2m_values_celsius.flatten(),
        'tp': tp_values_mm.flatten()
    })

    # Save DataFrame to CSV
    csv_filename = "climate_data_all_rows.csv"
    df.to_csv(csv_filename, index=False)

print(f"Climate data (all rows) saved to {csv_filename}")